# Step12 가변 길이 인수(개선편)
## 첫번째 개선 : 함수를 사용하게 쉽게

In [1]:
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      x, y = f.input, f.output 
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator) 

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

In [5]:
class Function:
  '''
    Function의 입력을 list말고, 가변길이 그 자체로 받아보자 : *args(가변길이 인수)
  '''
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]
    
    for output in outputs:
      output.set_creator(self) 
    
    self.inputs = inputs
    self.outputs= outputs 
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

In [6]:
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,) # tuple 형태로 반환

In [9]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


##12.2 두 번째 개선: 함수를 구현하기 쉽도록

In [10]:
class Function:
  '''
    가변 길이로 받은 inputs들을 unpack하고, 출력이 tuple 형태가 아닐 경우엔 추가 지원
  '''
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]
    
    for output in outputs:
      output.set_creator(self) 
    
    self.input = input
    self.output= output 
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

class Add(Function):
  '''
    사용하기 쉬운 직관적 형태로 변환!
  '''
  def forward(self, x0, x1):
    y = x0 + x1
    return y 

##12.3 add 함수 구현

In [11]:
def add(x0, x1):
  return Add()(x0, x1)

In [12]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5
